<a href="https://colab.research.google.com/github/quinn-dougherty/DS-Unit-2-Sprint-3-Advanced-Regression/blob/master/module1-logistic-regression/logisticregression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# main source: read.pudn.com/downloads158/ebook/702714/Larry Wasserman_ALL OF Statistics.pdf#page=243
# idk if this is the same edition i'm using. 
# i also used ESL (Hastie et al) and that's where the dataset came from, but that treatment is more useful for when there's more than 2 classes. 

dat_url = 'https://web.stanford.edu/~hastie/ElemStatLearn/datasets/SAheart.data'

'''A retrospective sample of males in a heart-disease high-risk region
of the Western Cape, South Africa. There are roughly two controls per
case of CHD. Many of the CHD positive men have undergone blood
pressure reduction treatment and other programs to reduce their risk
factors after their CHD event. In some cases the measurements were
made after these treatments. These data are taken from a larger
dataset, described in  Rousseauw et al, 1983, South African Medical
Journal. 

sbp		systolic blood pressure
tobacco		cumulative tobacco (kg)
ldl		low densiity lipoprotein cholesterol
adiposity
famhist		family history of heart disease (Present, Absent)
typea		type-A behavior
obesity
alcohol		current alcohol consumption
age		age at onset
chd		response, coronary heart disease
'''

import pandas as pd
import numpy as np 
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from numpy.testing import assert_almost_equal
from functools import reduce

In [102]:
df = pd.read_csv(dat_url).drop('row.names', axis=1)

assert all([x==0 for x in df.isna().sum().values])

df.head()


,sbp,tobacco,ldl,adiposity,famhist,typea,obesity,alcohol,age,chd
0,160,12.00,5.73,23.11,Present,49,25.30,97.20,52,1
1,144,0.01,4.41,28.61,Absent,55,28.87,2.06,63,1
2,118,0.08,3.48,32.28,Present,52,29.14,3.81,46,0
3,170,7.50,6.41,38.03,Present,51,31.99,24.26,58,1
4,134,13.60,3.50,27.78,Present,60,25.99,57.34,49,1


In [103]:
df.describe()

,sbp,tobacco,ldl,adiposity,typea,obesity,alcohol,age,chd
count,462.000000,462.000000,462.000000,462.000000,462.000000,462.000000,462.000000,462.000000,462.000000
mean,138.326840,3.635649,4.740325,25.406732,53.103896,26.044113,17.044394,42.816017,0.346320
std,20.496317,4.593024,2.070909,7.780699,9.817534,4.213680,24.481059,14.608956,0.476313
min,101.000000,0.000000,0.980000,6.740000,13.000000,14.700000,0.000000,15.000000,0.000000
25%,124.000000,0.052500,3.282500,19.775000,47.000000,22.985000,0.510000,31.000000,0.000000
50%,134.000000,2.000000,4.340000,26.115000,53.000000,25.805000,7.510000,45.000000,0.000000
75%,148.000000,5.500000,5.790000,31.227500,60.000000,28.497500,23.892500,55.000000,1.000000
max,218.000000,31.200000,15.330000,42.490000,78.000000,46.580000,147.190000,64.000000,1.000000


In [104]:
sclr = StandardScaler()

X = df.drop(['chd', 'famhist'], axis=1)
X['famhist'] = df.famhist.map({'Present': 1, 'Absent': 0})
X = sclr.fit_transform(X)
y = df.chd

X_train, X_test, y_train, y_test = train_test_split(X,y)

#logi_mod.classes_, logi_mod.coef_, logi_mod.intercept_

#help(logi_mod)

#logi_mod.predict(X)

''' https://stackoverflow.com/questions/52670012/convergencewarning-liblinear-failed-to-converge-increase-the-number-of-iterati

Normally when an optimization algorithm does not converge, it is usually because the problem is not well-conditioned, perhaps due to a poor scaling of the decision variables. There are a few things you can try.

    Normalize your training data so that the problem hopefully becomes more well conditioned, which in turn can speed up convergence. One possibility is to scale your data to 0 mean, unit standard deviation using Scikit-Learn's StandardScaler for an example. Note that you have to apply the StandardScaler fitted on the training data to the test data.
    Related to 1), make sure the other arguments such as regularization weight, C, is set appropriately.
    Set max_iter to a larger value. The default is 1000.

'''

logi_mod = LogisticRegression(solver='lbfgs', max_iter=1000).fit(X_train, y_train)



/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/usr/local/lib/python3.6/dist-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


In [106]:
assert_almost_equal(1 - np.divide(abs(logi_mod.predict(X_test) - y_test).sum(), X_test.shape[0]), 
                    np.divide(sum(logi_mod.predict(X_test) == y_test), X_test.shape[0])
                   )

for l in logi_mod.predict_proba(X_test):
  assert_almost_equal(l[0] + l[1], 1)
  
for l in logi_mod.predict_log_proba(X_test): 
  assert_almost_equal(np.exp(l[0])+np.exp(l[1]), 1)

print('error from discretized: {}'.format(1 - np.divide(abs(logi_mod.predict(X_test) - y_test).sum(), X_test.shape[0])))
#print('R2 (variance explained) (this is not R2/var explained): {}'.format(logi_mod.score(X_test, y_test)))
#print('error from continuous: ' )
predictions_proba = [l[0] for l in logi_mod.predict_proba(X_test)]

#1-np.divide(sum(1-abs(y_test.values - predictions_proba)), X_test.shape[0]) ## error from smooth'd? 

predictions_discr = logi_mod.predict(X_test)

mylist = np.array([x >= 0.5 for x in predictions_proba]).astype(int)

assert_almost_equal(np.divide(sum(logi_mod.predict(X_test) == y_test), X_test.shape[0]), 
                    1- np.divide(sum(mylist == y_test), X_test.shape[0]))



error from discretized: 0.646551724137931


1

In [0]:
X_ = df.drop(['chd', 'famhist'], axis=1)
X_['famhist'] = df.famhist.map({'Present': 1, 'Absent': 0})
y_ = df.chd

X_.shape, y_.shape

X = pd.concat([pd.DataFrame(np.ones(X_.shape[0]), columns=['ones']), pd.DataFrame(columns = X_.columns)], axis=1) 
for feat in X_.columns: 
  X[feat] = np.divide(X_[feat] - X_[feat].mean(), X_[feat].std())
  assert_almost_equal(X[feat].mean(), 0)
  assert_almost_equal(X[feat].std(), 1)
  
X.head() # standardization

beta = np.ones(X.shape[1]) # initialize, doesn't matter.

def y_hat(beta, dat=X): return np.matmul(dat, beta) 
# the basic linear combination that comprises the model in Linear Regression is the input to the squisher in logistic regression

def MSE(beta, dat=X): return np.divide(sum([(yy-yh)**2 for yy,yh in zip(y_,y_hat(beta, dat))]), dat.shape[0])
# mean squared error.... hm idk if this is even relevant. 


In [0]:
def logit(p): 
  assert 0<p<1
  return np.log(np.divide(p, 1-p))

def squish(x): 
  return np.divide(1, 1+np.exp(-x)) # ok this is 
# the standard sigmoid on wikipedia but 
# Wasserman suggests that the x isn't minus. 

assert_almost_equal(logit(squish(np.exp(-2))), np.exp(-2))
assert_almost_equal(logit(squish(np.exp(2))), np.exp(2))


### $\textbf{data}$ is a dataframe, all numeric and has been cleaned of nulls. Slice off one of it's features: this is our _dependent variable_. we have $N-1$ _observations_ and $M$ _independent features_

Our **dependent variable** is just 0 or 1. Two _classes_. 

we operate on that until we have $\textbf{X}$, a **dataframe of independent variables after standardization** of size $(N \times M)$,  a **column of ones** has been appended to the _leftmost_ side. 

The probability that observation $i$ is a member of class `1` (i.e. "has heart disease") is: $p = sigmoid(\textbf{X}\beta)$ where $\beta = (\beta_0, \beta_1, ... , \beta_n)$ and $i$ is in $(1, ... , n)$

Apply $logit$ to both sides, $ logit(p) = logit(sigmoid(\textbf{X}\beta))$, noticing that $logit(sigmoid(x))==x$, so your final equation is $logit(p) = \textbf{X} \beta $

where $p$ is a column of probabilities, equal in length to the number of observations. Each component of $p$ is the probability that it's corresponding observation has heart disease. I.e. if row $i$ of our data (before the test data $y$ had been sliced off, the standardization was applied, and the ones column was added) represents a person, we say "I am $p_i$ percent sure that person #$i$ has heart disease" (for any $i$ in $(0,..,n)$)

The _conditional likelihood_ is $\mathcal{L}(\beta) = \Pi_{i=1}^{n}(p_i(\beta)^{y_i}(1-p_i(\beta))^{1-y_i})$ (remember: $y_i$'s values are entirely in ${0,1}$!). Since we **prefer addition ${and}$ $\log$ is _monotonic_** (order preserving), we convert it to $\log(\mathcal{L}(\beta)) = \mathcal{l}(\beta) = \Sigma_{i=1}^{n}\log(p_i(\beta)^{y_i}(1-p_i(\beta))^{1-y_i})$. In other words, once you have probabilities of heart disease for each observation, simply add the $\log$ of the "has heart disease" probability if the _test data_ says "person #i has heart disease", and if the test data says "person #i doesn't have heart disease" then add the $\log$ of the negated probability, $1-p_i$.  

## The goal then is to find the $\beta$ that minimizes error, which is similar to maximizing the likelihood $(P(\textbf{data} | \beta)) = \mathcal{L}(\beta)) $

In [0]:
def p(beta, dat=X): 
  return squish(y_hat(beta, dat))


## luckily for us, this problem has been _analytically understood_, meaning there's nothing going on but an equation i can write down, provided i have the resources to matrically multiply. it's called the **reweighted least squares algorithm**

The equation is $\beta = (\textbf{X}^T\textbf{WX})^{-1}\textbf{X}^T\textbf{Wy}  $, where $\textbf{y}$ is training data and $\textbf{W}$ is a function of $\beta$

In [197]:
X_train, X_test, y_train, y_test = train_test_split(X,y_)

#
def update(beta, dat=X_train, depend=y_train): 
  pks = p(beta, dat)
  W = np.diag(pks*(1-pks))

  fact0 = np.matmul(dat.T, W)
  fact1 = np.matmul(fact0, dat)
  fact2 = np.linalg.inv(fact1)
  fact3 = np.matmul(fact2, fact0)
  return np.matmul(fact3, depend)


def repeatedly(f, n): 
  def composition(f,g):
    # "f after g" 
    return lambda x: f(g(x))
  return reduce(lambda ma, mma: composition(ma, mma), [f]*n)



update25 = repeatedly(update, 25)

update25(beta)

#MSE(beta_hat, X_train)

p(update25(beta))
# hrm, looking at these they're all wishy-washy! like it's madly uncertain! 

(p(update25(beta)) < 0.37).any() or (p(update25(beta)) > 0.73).any() # ugh

# logi_mod.predict_proba(X_test.drop('ones', axis=1))


array([[0.59941376, 0.40058624],
       [0.80303276, 0.19696724],
       [0.96079411, 0.03920589],
       [0.55555591, 0.44444409],
       [0.68295156, 0.31704844],
       [0.40474347, 0.59525653],
       [0.20526323, 0.79473677],
       [0.28438035, 0.71561965],
       [0.30018286, 0.69981714],
       [0.1583292 , 0.8416708 ],
       [0.83559228, 0.16440772],
       [0.35497416, 0.64502584],
       [0.74637724, 0.25362276],
       [0.57404503, 0.42595497],
       [0.83081611, 0.16918389],
       [0.22985571, 0.77014429],
       [0.97408342, 0.02591658],
       [0.44937014, 0.55062986],
       [0.80031558, 0.19968442],
       [0.36619583, 0.63380417],
       [0.32508194, 0.67491806],
       [0.24087165, 0.75912835],
       [0.77027345, 0.22972655],
       [0.89297523, 0.10702477],
       [0.7646139 , 0.2353861 ],
       [0.74853704, 0.25146296],
       [0.63092739, 0.36907261],
       [0.7212911 , 0.2787089 ],
       [0.5815017 , 0.4184983 ],
       [0.97638393, 0.02361607],
       [0.

In [175]:
# discretize the predictions with 0.5
my_discretized_predictions = np.array([x >= 0.5 for x in p(beta_hat, X_test)]).astype(int)

# assert_almost_equal(np.divide(sum(logi_mod.predict(X_test) == y_test), X_test.shape[0]), 
#                     1- np.divide(sum(mylist == y_test), X_test.shape[0]))

my_discretized_predictions.shape, y.shape
# assert_almost_equal(np.divide(sum(my_discretized_predictions==y_), X_.shape[0]), 
#                     1-np.divide(sum(my_discretized_predictions == y_), X_.shape[0]))


np.divide(sum(my_discretized_predictions == y_test), y_test.shape[0])




0.5517241379310345

_definition_

$\textbf{f is monotonic} \iff x \leq y \implies f x \leq f y$ 

whenever $f$'s domain and codomain each have partial orders. 